In [9]:
import pandas as pd

df = pd.read_csv('./data/cabaventa.csv')
df.shape

(183810, 27)

## Limpieza Texto y Columnas

En esta parte se realiza una limpieza básica con `regex` de algunas columnas a los fines de normalizar un poco el texto: lowering, punctuation, spaces, digits. La función armada permite customizar la limpieza, por ende es posible no aplicar la misma limpieza para todos los casos. La función genera columnas nuevas con el nombre`f"{col}_cleaned"`  

Dado que las columnas textuales `title` y `description` no fueron utilizadas en la predicción ni analizadas mediante técnicas de embeddings o BOW, no se trataron los NaN en esas columnas. 

In [10]:
from cleaning import CleaningData
#instanciamos Cleaning Data con df

cleaned = CleaningData(data=df)

In [11]:
#Quitamos columnas 
cols_to_drop = ['Unnamed: 0.1', 'Unnamed: 0', #columnas que vinieron mal en la bajada
                'operation_type', #todos son 'venta'
                'l1', #todas las filas iguales 
                'l2', #todas las filas iguales 
                'ad_type', #todas las filas iguales 
                'l4', #todas las filas nulas preferible el dato de lat y long
                'l5', #columna con todas las filas nulas
                'l6', #columna con todas las filas nulas
                'created_on', #la columna created_on es igual a la columna start_date
                'price_period' #la columna contiene muchos NaN y un único valor
               ]
data = cleaned.drop_columns(columns=cols_to_drop)
data.shape

Cleaning columns with no valuable information...



(183810, 16)

In [12]:
#Limpieza title
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="title", params=params)
data.shape

(183810, 17)

In [13]:
#Limpieza description
params = {'lowering':True,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="description", params=params)
data.shape

(183810, 18)

In [14]:
#Limpieza texto de l3
params = {'lowering':False,'punctuation':True,'accents': True,'strip_spaces':True,'digits':False,'within_spaces':True}
cleaned.text_col_cleaning(text_column="l3", params=params)
data.shape

(183810, 19)

## Barrio, latitud y longitud

#### Valores nulos

In [26]:
(data[['lat','lon','l3_cleaned']].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

lat           6.23%
lon           6.23%
l3_cleaned    1.00%
dtype: object

Se detectó una mayor presencia de valores nulos en los datos de georreferenciación que en el dato del barrio. Por ende se realizó un trabajo de imputación de nulos partiendo de considerar a la columna `l3_cleaned` con una mayor validez que la columnas `lat` y `lon`. Por ende, fue tomada como "eje" para realizar imputación de nulos. 

#### Errores

* Barrios cuya denomiación no es la denominación oficial: Ejemplo Centro / Microcentro, Las Cañitas, Pompeya, Abasto, Once, etc. 

In [25]:
data.l3_cleaned.unique()

array(['Floresta', 'Belgrano', 'Recoleta', 'Monserrat', 'San Nicolas',
       'Retiro', 'Almagro', 'Palermo', 'Saavedra', 'Chacarita',
       'Barrio Norte', 'Villa Crespo', 'Villa Pueyrredon', 'Flores',
       'Villa Urquiza', 'Once', 'Coghlan', 'Congreso', 'Villa Devoto',
       'Boedo', 'Nunez', 'Barracas', 'Colegiales', 'Villa del Parque',
       'Caballito', 'Balvanera', 'San Telmo', 'Monte Castro',
       'Villa General Mitre', 'Parque Chacabuco', None, 'Abasto',
       'Puerto Madero', 'Parque Chas', 'San Cristobal',
       'Villa Santa Rita', 'Villa Lugano', 'Liniers', 'Agronomia',
       'Villa Ortuzar', 'Villa Soldati', 'Villa Luro',
       'Parque Centenario', 'Las Canitas', 'Paternal',
       'Centro Microcentro', 'Mataderos', 'Parque Avellaneda', 'Pompeya',
       'Parque Patricios', 'Tribunales', 'Boca', 'Villa Real',
       'Constitucion', 'Versalles', 'Catalinas', 'Velez Sarsfield',
       'Villa Riachuelo'], dtype=object)

* Columna `lat` con valores fuera del polígono de CABA

In [27]:
data.lat.describe()

count    172362.000000
mean        -34.590007
std           0.376025
min         -53.788624
25%         -34.615663
50%         -34.596872
75%         -34.577306
max          29.753374
Name: lat, dtype: float64

* Columna `lon` con valores fuera de los valores correctos. 

In [28]:
data.lon.describe()

count    172362.000000
mean        -58.462583
std           0.514948
min        -100.469651
25%         -58.464658
50%         -58.439264
75%         -58.408084
max          -5.490771
Name: lon, dtype: float64

#### Procesamiento realizado

Se establecieron los siguientes pasos: 
1) Normalización datos barrios: dado que eran pocos casos se realizó una imputación manual sobre la columna `l3_cleaned`. Ej: Las Canitas ---> Palermo. Pompeya --> Nueva Pompeya. Abasto --> Almagro.
2) Si la latitud, longitud fueron NaN o localización fuera de CABA se completó con un punto centroide del barrio, si para dicho registro el dato del barrio fue nulo entonces se quitó toda la fila. 
3) Si el barrio fue nulo pero la localización dentro de CABA, se completó barrio verificando en qué polígono pertencía el punto.
4) Si el barrio y la localización no son nulos pero sí incongruentes (localización pertence a otro barrio) se procedío a imputar el punto centroide del barrio. 

Pendiente para un próximo abordaje: 

* Para los casos en que la localización era externa a CABA, el barrio no nulo, se imputó un punto centroide del barrio. Eso pudo conducir a errores dado que implícitamente se "confía" en que el dato del barrio fue correcto. Una alternativa sería probar adicionalmente si de la columna `description_cleaned` fuera posible confirmar que no se tratase de casos que efectivamente pertenecieran a localizaciones externas a CABA. 

ACA VA LA MAGIA DE LUCASSSSSS!!!!

## Superficie total, superficie cubierta, ambientes, dormitorios y precio

#### Valores nulos

In [32]:
cols = ["surface_total","surface_covered","rooms","bedrooms", "price"]
(data[cols].isna().sum()/len(data)).apply(lambda x: "{:.2%}".format(x))

surface_total      33.13%
surface_covered    34.02%
rooms              15.30%
bedrooms           27.19%
price               1.85%
dtype: object

Dada la alta correlación entre `surface_total` y `surface_covered` y entre `rooms` y `bedrooms`, para los casos en que el par tuviera datos nulos se procedió a eliminar la fila ante la dificultad para imputar nulos en esos casos. 

```
data.dropna(subset = ['rooms', bedrooms'])
data.dropna(subset = ['surface_total', 'surface_covered'])
```

Para la variable `price` se eliminaron las filas que tenían nulos ya que por ser la variable sobre la cual se constuirá la variable respuesta se decidió no realizar imputación de valores faltantes sobre la misma. 

```
data.dropna(subset =['price'])
```

#### Errores / Outliers

Se detecton que las columnas contenían errores o valores outliers. Se enumeran a continuación: 

* `surface_total < surface_covered`

In [33]:
(data.surface_total < data.surface_covered).sum() 

926

* outlieres en `surface_total`

In [35]:
data.surface_total.describe()

count    122908.00000
mean        174.63733
std        1461.00811
min          10.00000
25%          45.00000
50%          70.00000
75%         127.00000
max      140380.00000
Name: surface_total, dtype: float64

* outliers en `surface_covered`

In [36]:
data.surface_covered.describe()

count    121282.000000
mean        148.706692
std        3064.441474
min           1.000000
25%          41.000000
50%          60.000000
75%         103.000000
max      950000.000000
Name: surface_covered, dtype: float64

* `rooms < bedrooms`

In [34]:
(data.rooms < data.bedrooms).sum() 

590

* outliers en `rooms`

In [37]:
data.rooms.describe()

count    155680.000000
mean          2.706719
std           1.520259
min           1.000000
25%           2.000000
50%           3.000000
75%           4.000000
max          40.000000
Name: rooms, dtype: float64

* outliers en `bedrooms`

In [ ]:
data.bedrooms.describe()

count    133834.000000
mean          1.983696
std           2.327565
min          -2.000000
25%           1.000000
50%           2.000000
75%           3.000000
max         286.000000
Name: bedrooms, dtype: float64

#### Procesamiento realizado

1) Borrar filas donde los pares de columnas `['rooms','bedrooms']` o `['surface_total','surface_covered']` contengan nulos. 
2) Intercambiar los valores de las columnas cuando `rooms < bedrooms` o `surface_total < surface_covered`. 
3) Eliminar filas donde `price` sea nulo. 
4) Si para una fila uno de los valores de `['surface_total', 'surface_covered']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
5) Si para una fila uno de los valores de `['rooms', bedrooms']` contiene valor outlier y el otro no es outlier, se le imputó un `NaN` al dato outlier y se dejó dicho valor para ser imputado luego por método MICE. En caso que los dos valores fueran outliers o uno outlier y el otro `NaN` se borró la fila. 
6) Se borraron los valores outliers de `price`
7) Luego de estos pasos se procedió a imputar los valores nulos con el método multivariado MICE en las columnas `['rooms', bedrooms','surface_total','surface_covered']` utilizando también la variable `price` como predictor. 

## Modificación de los tipos de datos

Acá hay que señalar si se realizaron cambios en los tipos de datos: ejemplo todas las variables que se pasan a categoricas para mejorar el uso de memoria que realiza pandas. 

## Resultados Pre-procesamiento. 

Enumerar cantidad de filas y columnas iniciales, enumerar cantidad de filas y columnas resultantes. 